# Step 1. Import Libraries

In [74]:
import pandas as pd
import geopandas as gpd
import numpy as np
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
%matplotlib inline
import altair as alt
import hvplot.pandas

In [86]:
from geopandas import GeoDataFrame 

# Step 2. Get Data

## Step 2.1 Neighborhood Boundaries 

In [16]:
# load the Zillow data from GitHub
neighborhood = gpd.read_file("C:/Users/laqan/Downloads/620/final/data/Neighborhood Tabulation Areas.geojson")

# convert the CRS to Web Mercator
neighborhood  = neighborhood .to_crs(epsg=3857)

## Step 2.2 Census Data 

In [39]:
census_tract = gpd.read_file("C:/Users/laqan/Downloads/620/final/data/2010 Census Tracts.geojson")

In [62]:
census_data = pd.read_csv("C:/Users/laqan/Downloads/620/final/data/census_data.csv")

In [63]:
census_data['boro_ct2010']=census_data['boro_ct2010'].astype(str)

In [90]:
joined_data = census_tract.merge(census_data, on='boro_ct2010')

## Step 2.3 Join Data 

In [93]:
joined_data = joined_data.to_crs(epsg=3857)

In [97]:
# perform the spatial join
gdata = gpd.sjoin(joined_data, neighborhood, op='intersects', how='left')

In [107]:
gdata_group = gdata.groupby(['ntaname_right']).sum()

In [110]:
gdata_group =gdata_group.add_suffix('_sum').reset_index()

In [113]:
gdata_group = gdata_group[['ntaname_right','B01001_001E_sum', 'ukrainian_sum', 'turkish_sum', 'romanian_sum', 'russian_sum', 'polish_sum', 'german_sum', 'italian_sum', 'french_sum', 'arabian_sum']]

In [115]:
gdata_group.rename(columns={'ntaname_right':'ntaname'}, inplace=True)

D:\Anaconda\envs\musa-620\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [117]:
neighborhood_data = neighborhood.merge(gdata_group, on='ntaname')

# Step 3. Plot 

In [165]:
import folium
import osmnx as ox
import networkx as nx
import matplotlib.colors as mcolors
from folium.plugins import HeatMap
from IPython.display import IFrame
import requests
import json
import matplotlib.colors as mcolors
from xml.etree import ElementTree
from folium.plugins import MarkerCluster
from folium.map import *

## Step 3.1 Normalize the data column to 0 to 1 

In [153]:
neighborhood_data['pct_russian'] = neighborhood_data['russian_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_ukrainian'] = neighborhood_data['ukrainian_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_turkish'] = neighborhood_data['turkish_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_romanian'] = neighborhood_data['romanian_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_polish'] = neighborhood_data['polish_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_german'] = neighborhood_data['german_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_italian'] = neighborhood_data['italian_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_french'] = neighborhood_data['french_sum'] / neighborhood_data['B01001_001E_sum']
neighborhood_data['pct_arabian'] = neighborhood_data['arabian_sum'] / neighborhood_data['B01001_001E_sum']

In [155]:
x = neighborhood_data['pct_russian']
neighborhood_data['pct_russian_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_ukrainian']
neighborhood_data['pct_ukrainian_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_turkish']
neighborhood_data['pct_turkish_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_romanian']
neighborhood_data['pct_romanian_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_polish']
neighborhood_data['pct_polish_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_german']
neighborhood_data['pct_german_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_italian']
neighborhood_data['pct_italian_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_french']
neighborhood_data['pct_french_normalized'] = (x - x.min()) / (x.max() - x.min())

x = neighborhood_data['pct_arabian']
neighborhood_data['pct_arabian_normalized'] = (x - x.min()) / (x.max() - x.min())


In [157]:
# use a red-purple colorbrewer color scheme
def get_highlighted_style(feature):
    return {'weight': 2, 'color': 'red'}

colorbar = plt.get_cmap('RdPu')

def get_style_russian(feature):
    value = feature['properties']['pct_russian_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_ukrainian(feature):
    value = feature['properties']['pct_ukrainian_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_romanian(feature):
    value = feature['properties']['pct_romanian_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_polish(feature):
    value = feature['properties']['pct_polish_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_german(feature):
    value = feature['properties']['pct_german_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_italian(feature):
    value = feature['properties']['pct_italian_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_french(feature):
    value = feature['properties']['pct_french_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

def get_style_arabian(feature):
    value = feature['properties']['pct_arabian_normalized']
    rgb_color = colorbar(value)
    color = mcolors.rgb2hex(rgb_color)
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

In [166]:
# initialize the map
m = folium.Map(location=[40.733189, -73.932258], zoom_start=11, tiles='cartodbpositron')

# add the GeoJson to the map
folium.GeoJson(
    neighborhood_data,
    name='russian',
    style_function=get_style_russian,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_russian'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='ukrainian',
    style_function=get_style_ukrainian,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_ukrainian'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='romanian',
    style_function=get_style_romanian,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_romanian'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='polish',
    style_function=get_style_polish,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_polish'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='german',
    style_function=get_style_german,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_german'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='italian',
    style_function=get_style_italian,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_italian'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='french',
    style_function=get_style_french,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_french'])
).add_to(m)

folium.GeoJson(
    neighborhood_data,
    name='arabian',
    style_function=get_style_arabian,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['ntaname', 'pct_arabian'])
).add_to(m)


folium.LayerControl().add_to(m)

# avoid a rendering bug by saving as HTML and re-loading
m.save('D:/map.html')

TypeError: add_children() got an unexpected keyword argument 'style_function'